<img src="https://www.bestdesigns.co/uploads/inspiration_images/4350/990__1511457498_404_walmart.png" alt="WALMART LOGO" />

# Walmart : predict weekly sales

## Company's Description 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## Goals 🎯

The project can be divided into three steps:

- Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2 : train a **linear regression model** (baseline)
- Part 3 : avoid overfitting by training a **regularized regression model**

## Scope of this project 🖼️

For this project, you'll work with a dataset that contains information about weekly sales achieved by different Walmart stores, and other variables such as the unemployment rate or the fuel price, that might be useful for predicting the amount of sales. The dataset has been taken from a Kaggle competition, but we made some changes compared to the original data. Please make sure that you're using **our** custom dataset (available on JULIE). 🤓

## Deliverable 📬

To complete this project, your team should: 

- Create some visualizations
- Train at least one **linear regression model** on the dataset, that predicts the amount of weekly sales as a function of the other variables
- Assess the performances of the model by using a metric that is relevant for regression problems
- Interpret the coefficients of the model to identify what features are important for the prediction
- Train at least one model with **regularization (Lasso or Ridge)** to reduce overfitting


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you: 

### Part 1 : EDA and data preprocessing

Start your project by exploring your dataset : create figures, compute some statistics etc...

Then, you'll have to make some preprocessing on the dataset. You can follow the guidelines from the *preprocessing template*. There will also be some specific transformations to be planned on this dataset, for example on the *Date* column that can't be included as it is in the model. Below are some hints that might help you 🤓

 #### Preprocessing to be planned with pandas

 **Drop lines where target values are missing :**
 - Here, the target variable (Y) corresponds to the column *Weekly_Sales*. One can see above that there are some missing values in this column.
 - We never use imputation techniques on the target : it might create some bias in the predictions !
 - Then, we will just drop the lines in the dataset for which the value in *Weekly_Sales* is missing.
 
**Create usable features from the *Date* column :**
The *Date* column cannot be included as it is in the model. Either you can drop this column, or you will create new columns that contain the following numeric features : 
- *year*
- *month*
- *day*
- *day of week*

**Drop lines containing invalid values or outliers :**
In this project, will be considered as outliers all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. This concerns the columns : *Temperature*, *Fuel_price*, *CPI* and *Unemployment*
 


**Target variable/target (Y) that we will try to predict, to separate from the others** : *Weekly_Sales*

 **------------**

 #### Preprocessings to be planned with scikit-learn

 **Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

 - Categorical variables : Store, Holiday_Flag
 - Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek

### Part 2 : Baseline model (linear regression)
Once you've trained a first model, don't forget to assess its performances on the train and test sets. Are you satisfied with the results ?
Besides, it would be interesting to analyze the values of the model's coefficients to know what features are important for the prediction. To do so, the `.coef_` attribute of scikit-learn's LinearRegression class might be useful. Please refer to the following link for more information 😉 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

### Part 3 : Fight overfitting
In this last part, you'll have to train a **regularized linear regression model**. You'll find below some useful classes in scikit-learn's documentation :
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

**Bonus question**

In regularized regression models, there's a hyperparameter called *the regularization strength* that can be fine-tuned to get the best generalized predictions on a given dataset. This fine-tuning can be done thanks to scikit-learn's GridSearchCV class : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Also, you'll find here some examples of how to use GridSearchCV together with Ridge or Lasso models : https://alfurka.github.io/2018-11-18-grid-search/

### --- Import ---

In [1]:
### 1 - library import ### ----

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


### --- File reading and basic exploration ---

In [2]:
#file reading and basic exploration - import dataset 

# read data
data = pd.read_csv("Walmart_Store_sales.csv")


In [3]:
#file reading and basic exploration - get basic stats 

# print shape of data
print("Number of rows: {}".format(data.shape[0]))
print("Number of columns: {}".format(data.shape[1]))
print()

# display dataset
pd.set_option('display.max_columns', None)
print("Dataset display: ")
display(data.head())
print()

# display basic statistics
print("Basics statistics: ")
data_desc = data.describe(include='all')
display(data_desc)
print()


Number of rows: 150
Number of columns: 8

Dataset display: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [4]:
#file reading and basic exploration 
#get percentage of missing values

# check wether some columns are full of NaNs
column_nan_full = data.columns[data.isnull().all()]
column_nb = len(column_nan_full)

# get percentage of missing values in columns
percent_nan_col = data.isnull().sum() / data.shape[0] * 100

# check wether some rows are full of NaNs
row_nan_count = pd.Series([data.loc[i,:].isnull().sum() for i in range(0, data.shape[0])])
row_nan_full = row_nan_count.index[row_nan_count == data.shape[1]]
row_nb = len(row_nan_full)

# print report
print("COLUMNS")
print("{} columns out of {} are fully filled with missing values".format(column_nb,data.shape[1]))
print("Percentage of missing values per column:\n{}".format(percent_nan_col))
print()
print("ROWS")
print("{} rows out of {} are fully filled with missing values".format(row_nb,data.shape[0]))


COLUMNS
0 columns out of 8 are fully filled with missing values
Percentage of missing values per column:
Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

ROWS
0 rows out of 150 are fully filled with missing values


### --- Exploratory data analysis ---

In [5]:
# exploratory data analysis - plot univariate analysis 

# set figure to make subplots
fig1 = make_subplots(
    rows = 2,
    cols = 4,
    subplot_titles = (
        "A. Weekly sales",
        "B. Temperature",
        "C. Fuel price",
        "D. CPI",
        "E. Unemployment rate",
        "F. Holidays",
        "G. Stores"),
    column_widths = [0.20, 0.20, 0.20, 0.20],
    horizontal_spacing = 0.15)

# plot distribution of each numeric variable
features_num = ["Weekly_Sales", "Temperature", "Fuel_Price", "CPI", "Unemployment"]
[fig1.add_trace(go.Histogram(
    x = data[features_num[i]],
    marker_color = px.colors.qualitative.Vivid[i]),
    row = 1, col = i+1) for i in [0, 1, 2, 3]]
[fig1.add_trace(go.Histogram(
    x = data[features_num[i]],
    marker_color = px.colors.qualitative.Vivid[i]),
    row = 2, col = i-3) for i in [4]]

# plot categorical variables
holidays = data["Holiday_Flag"].value_counts()
fig1.add_trace(go.Bar(
    x = ["No", "Yes"],
    y = holidays.values,
    marker_color = px.colors.qualitative.Vivid[6:]),
    row = 2, col = 2)
stores = data["Store"].value_counts()
fig1.add_trace(go.Bar(
    x = stores.index,
    y = stores.values,
    marker_color = px.colors.qualitative.Vivid[8]),
    row = 2, col = 3)

# update layout
fig1.update_annotations(font_size = 15)
fig1.update_xaxes(tickfont = dict(size = 10))
fig1.update_yaxes(tickfont = dict(size = 10))
fig1.update_layout(
        margin = dict(l = 90, t= 120),
        title_text = "Figure 1. Univariate analysis",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = go.layout.xaxis.Title(text = "Weekly sales (Dollars)", font_size = 10)),
        xaxis2 = dict(title = go.layout.xaxis.Title(text = "Temperature (Degrees Fahrenheit)", font_size = 10)),
        xaxis3 = dict(title = go.layout.xaxis.Title(text = "Fuel price (Dollars)", font_size = 10)),
        xaxis4 = dict(title = go.layout.xaxis.Title(text = "CPI (AU)", font_size = 10)),
        xaxis5 = dict(title = go.layout.xaxis.Title(text = "Unemployment rate (Percent)", font_size = 10)),
        xaxis6 = dict(title = go.layout.xaxis.Title(text = "Holiday period", font_size = 10)),
        xaxis7 = dict(title = go.layout.xaxis.Title(text = "Store id", font_size = 10)),
        yaxis = dict(range = [0, 24], tickvals = [0, 5, 10, 15, 20]),
        yaxis2 = dict(range = [0, 24], tickvals = [0, 5, 10, 15, 20]),
        yaxis3 = dict(range = [0, 36], tickvals = [0, 10, 20, 30]),
        yaxis4 = dict(range = [0, 72], tickvals = [0, 20, 40, 60]),
        yaxis5 = dict(range = [0, 30], tickvals = [0, 5, 10, 15, 20, 25]),
        yaxis6 = dict(title = go.layout.yaxis.Title(text = "Count", font_size = 10, standoff = 0), 
            range = [0, 180], tickvals = [0, 50, 100, 150]),
        yaxis7 = dict(title = go.layout.yaxis.Title(text = "Count", font_size = 10, standoff = 0), 
            range = [0, 18], tickvals = [0, 5, 10, 15]),
        bargroupgap = 0.4,
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 600)

fig1.show()


In [6]:
#exploratory data analysis - plot bivariate analysis

# plot pairwise dependencies
fig2 = go.Figure(data = go.Splom(
        dimensions = [dict(label = "Weekly sales", values = data["Weekly_Sales"]),
                dict(label = "Holidays", values = data["Holiday_Flag"]),
                dict(label = "Temperature", values = data["Temperature"]),
                dict(label = "Fuel price", values = data["Fuel_Price"]),
                dict(label = "CPI", values = data["CPI"]),
                dict(label = "Unemployment", values = data["Unemployment"]),
                dict(label = "Stores", values = data["Store"])],
        marker = dict(color = px.colors.qualitative.Vivid[1], size = 5),
        diagonal = dict(visible = False)))

# update layout
fig2.update_layout(
        margin = dict(l = 105, t = 100),
        title_text = "Figure 2. Bivariate analysis",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,   
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 800)

fig2.show()


In [7]:
# Exploratory data analysis - plot correlation matrix 

# get correlation matrix
corr_matrix = data.loc[:,features_num].corr().round(2)

# plot correlation matrix
fig3 = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())

# update layout
fig3.update_layout(
        margin = dict(l = 150, b = 40),
        title_text = "Figure 3. Correlation matrix",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,   
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig3.show()


### --- Preprocessing ---

In [8]:
# Preprocessing - Cleaning

# copy data for safety
data1 = data.copy()


# drop rows where values are missing (for date, weekly sales and holiday flag)

# get index of data to drop
mask_drop = (data1["Date"].isnull()) | (data1["Weekly_Sales"].isnull()) | (data1["Holiday_Flag"].isnull())
index_drop = data1.index[mask_drop]

# drop data
data1 = data1.drop(index_drop, axis = 0)

# print report
print("Number of rows with missing values that were dropped: {}"
    .format(len(index_drop)))
print()

# create usable features from the Date column and drop date

# format date column
data1["Date"] = pd.to_datetime(data1["Date"], infer_datetime_format = True)

# create features year, month, day, day of week
data1["year"] = data1["Date"].dt.year
data1["month"] = data1["Date"].dt.month
data1["day"] = data1["Date"].dt.day
data1["day_of_week"] = data1["Date"].dt.day_of_week
data1 = data1.drop(["Date"], axis = 1)

# display new dataset
print("Dataset display: ")
display(data1.head())
print("Data shape: {}".format(data1.shape))
print()


# drop rows containing outliers

# set columns to check
columns_tocheck = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

# initialise variables to store number of rows dropped
drop_nb = 0

# loop through columns
for i in columns_tocheck:

    # set bounds to identify outliers
    column_mean = data1[i].mean()
    column_std = data1[i].std()
    lower_bond = column_mean - 3 * column_std
    upper_bond = column_mean + 3 * column_std

    # get index of rows to drop
    mask_drop = (data1[i] < lower_bond) | (data1[i] > upper_bond)
    index_drop = data1.index[mask_drop]

    # drop rows
    data1 = data1.drop(index_drop, axis = 0)

    # count rows that were droped
    drop_nb += len(index_drop)

# print report
print("Number of rows with outliers that were dropped: {}".format(drop_nb))


Number of rows with missing values that were dropped: 41

Dataset display: 


C:\Users\Rafik\AppData\Local\Temp\ipykernel_15968\2443720797.py:24: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Rafik\AppData\Local\Temp\ipykernel_15968\2443720797.py:24: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_week
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25,4
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28,4
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010,5,28,4
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011,6,3,4
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012,2,3,4


Data shape: (109, 11)

Number of rows with outliers that were dropped: 5


In [9]:
# Preprocessing - Process data for  machine learning 

# separate target variable Y from features X
X = data1.drop(["Weekly_Sales"], axis = 1)
Y = data1["Weekly_Sales"]

# divide dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# create preprocessor object from pipelines for numeric and categorical features
features_num = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "year", "month", "day", "day_of_week"]
features_cat = ["Store", "Holiday_Flag"]
numeric_transformer = Pipeline(steps = [
    ("imputer", KNNImputer()),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps = [
    ("encoder", OneHotEncoder(drop = "first"))
])
preprocessor = ColumnTransformer(transformers = [
    ("num", numeric_transformer, features_num),
    ("cat", categorical_transformer, features_cat)
])

# impute missing values and scale numeric features, encode categorical features
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


### --- Baseline model (linear regression) ---

In [10]:
# train model
regressor1 = LinearRegression()
regressor1.fit(X_train, Y_train)

# make predictions on train and test sets
Y_train_pred1 = regressor1.predict(X_train)
Y_test_pred1 = regressor1.predict(X_test)

# perform 5-fold cross-validation to evaluate the generalized R2 score 
scores1 = cross_val_score(regressor1, X_train, Y_train, cv = 5)
print('Cross-validated R2 score: ', scores1.mean())
print('Standard deviation: ', scores1.std())
print()

# assess performance and print report
r2_train1 = r2_score(Y_train, Y_train_pred1)
r2_test1 = r2_score(Y_test, Y_test_pred1)
print("R2 score on training set: ", r2_train1)
print("R2 score on test set: ", r2_test1)


Cross-validated R2 score:  0.9189465722138006
Standard deviation:  0.047384243589428034

R2 score on training set:  0.9760547404425668
R2 score on test set:  0.9298924682807932


### --- Feature importance ---

In [11]:
# get column names from the preprocessor
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: 
    if name == 'num': 
        features = features_list 
    else: 
        features = pipeline.named_steps['encoder'].get_feature_names_out() 
    column_names.extend(features)

# store coefficients in a dataframe
coefs1 = pd.DataFrame(index = range(0,len(regressor1.coef_)), columns = ["features", "coefficients"])
coefs1["features"] = column_names
coefs1["coefficients"] = abs(regressor1.coef_)

# get feature importance
feature_importance1 = coefs1.sort_values("coefficients", ascending = False).reset_index(drop = True)

# plot feature importance
fig4 = go.Figure([go.Bar(
    x = feature_importance1.loc[:,"features"],
    y = feature_importance1.loc[:,"coefficients"],
    marker_color = px.colors.qualitative.Vivid)])

# update layout
fig4.update_xaxes(tickfont = dict(size = 10), tickangle = 90)
fig4.update_yaxes(tickfont = dict(size = 10))
fig4.update_layout(
        margin = dict(l = 120),
        title_text = "Figure 4. Baseline model feature importance",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = "Features"),
        yaxis = dict(title = "Coefficients", range = [-50000, 1600000], tickvals = [0, 500000,1000000,1500000]),
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig4.show()


# --- Regularized regression model (Ridge) ---

In [12]:
# train model
regressor2 = Ridge()
regressor2.fit(X_train, Y_train)

# make predictions on train and test sets
Y_train_pred2 = regressor2.predict(X_train)
Y_test_pred2 = regressor2.predict(X_test)

# perform 5-fold cross-validation to evaluate the generalized R2 score 
scores2 = cross_val_score(regressor2, X_train, Y_train, cv = 5)
print('Cross-validated R2 score: ', scores2.mean())
print('Standard deviation: ', scores2.std())
print()

# assess performance and print report
r2_train2 = r2_score(Y_train, Y_train_pred2)
r2_test2 = r2_score(Y_test, Y_test_pred2)
print("R2 score on training set: ", r2_train2)
print("R2 score on test set: ", r2_test2)


Cross-validated R2 score:  0.8316703281806719
Standard deviation:  0.05541776428192827

R2 score on training set:  0.9305744290211542
R2 score on test set:  0.8660674720747636


# --- Features importance ---

In [13]:
# store coefficients in a dataframe
coefs2 = pd.DataFrame(index = range(0,len(regressor2.coef_)), columns = ["features", "coefficients"])
coefs2["features"] = column_names
coefs2["coefficients"] = abs(regressor2.coef_)

# get feature importance
feature_importance2 = coefs2.sort_values("coefficients", ascending = False).reset_index(drop = True)

# plot feature importance
fig5 = go.Figure([go.Bar(
    x = feature_importance2.loc[:,"features"],
    y = feature_importance2.loc[:,"coefficients"],
    marker_color = px.colors.qualitative.Vivid)])

# update layout
fig5.update_xaxes(tickfont = dict(size = 10), tickangle = 90)
fig5.update_yaxes(tickfont = dict(size = 10))
fig5.update_layout(
        margin = dict(l = 120),
        title_text = "Figure 5. Regularized model feature importance",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = "Features"),
        yaxis = dict(title = "Coefficients", range = [-50000, 1600000], tickvals = [0, 500000,1000000,1500000]),
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig5.show()


# --- Regularized model fine-tuning ---

In [14]:
# tune lambda with gridsearch
params = {
    'alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0]
}
gridsearch = GridSearchCV(regressor2, param_grid = params, cv = 5)
gridsearch.fit(X_train, Y_train)

# print report
print("Best hyperparameter: ", gridsearch.best_params_)
print()

# train model
regressor3 = Ridge(alpha = gridsearch.best_params_["alpha"])
regressor3.fit(X_train, Y_train)

# perform 5-fold cross-validation to evaluate the generalized R2 score 
scores3 = cross_val_score(regressor3, X_train, Y_train, cv = 5)
print('Cross-validated R2 score: ', scores3.mean())
print('Standard deviation: ', scores3.std())
print()

# make predictions on train and test sets
Y_train_pred3 = regressor3.predict(X_train)
Y_test_pred3 = regressor3.predict(X_test)

# assess performance and print report
r2_train3 = r2_score(Y_train, Y_train_pred3)
r2_test3 = r2_score(Y_test, Y_test_pred3)
print("R2 score on training set: ", r2_train3)
print("R2 score on test set: ", r2_test3)


Best hyperparameter:  {'alpha': 0.01}

Cross-validated R2 score:  0.9195060778557547
Standard deviation:  0.04718100083732717

R2 score on training set:  0.9759939174302046
R2 score on test set:  0.9308389741172691


# --- Feature importance after tuning ---

In [15]:
# store coefficients in a dataframe
coefs3 = pd.DataFrame(index = range(0,len(regressor3.coef_)), columns = ["features", "coefficients"])
coefs3["features"] = column_names
coefs3["coefficients"] = abs(regressor3.coef_)

# get feature importance
feature_importance3 = coefs3.sort_values("coefficients", ascending = False).reset_index(drop = True)

# plot feature importance
fig6 = go.Figure([go.Bar(
    x = feature_importance3.loc[:,"features"],
    y = feature_importance3.loc[:,"coefficients"],
    marker_color = px.colors.qualitative.Vivid)])

# update layout
fig6.update_xaxes(tickfont = dict(size = 10), tickangle = 90)
fig6.update_yaxes(tickfont = dict(size = 10))
fig6.update_layout(
        margin = dict(l = 120),
        title_text = "Figure 6. Regularized model feature importance after tuning",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,
        xaxis = dict(title = "Features"),
        yaxis = dict(title = "Coefficients", range = [-50000, 1600000], tickvals = [0, 500000,1000000,1500000]),
        showlegend = False,
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig6.show()


# --- Results ---

In [16]:
# store results in a dataframe
results = pd.DataFrame(index = ["Linear regression", "Ridge lambda=1", "Ridge lambda=0.01"], 
    columns = ["Cross-validated R2", "R2 standard deviation", "R2 train", "R2 test"])
results["Cross-validated R2"] = [scores1.mean(), scores2.mean(), scores3.mean()]
results["R2 standard deviation"] = [scores1.std(), scores2.std(), scores3.std()]
results["R2 train"] = [r2_train1, r2_train2, r2_train3]
results["R2 test"] = [r2_test1, r2_test2, r2_test3]

# print results summary
print("Results summary: ")
display(results)


Results summary: 


,Cross-validated R2,R2 standard deviation,R2 train,R2 test
Linear regression,0.918947,0.047384,0.976055,0.929892
Ridge lambda=1,0.831670,0.055418,0.930574,0.866067
Ridge lambda=0.01,0.919506,0.047181,0.975994,0.930839


# --- Store selection and economic indicators ---

In [17]:
# copy data for safety
data2 = data1.copy()

# get top 6 stores with the highest coefficients from model3 (coefficients above 1M)
stores_desc = [5.0, 3.0, 9.0, 16.0, 7.0, 15.0]

# extract data for these stores
features_totest = ["Weekly_Sales", "Temperature", "Fuel_Price", "CPI", "Unemployment"]
data2 = data2.loc[data2["Store"].isin(stores_desc),features_totest]

# get correlation matrix
corr_matrix2 = data2.corr().round(2)

# plot correlation matrix
fig7 = ff.create_annotated_heatmap(corr_matrix2.values,
                                  x = corr_matrix2.columns.tolist(),
                                  y = corr_matrix2.index.tolist())

# update layout
fig7.update_layout(
        margin = dict(l = 150, b = 40),
        title_text = "Figure 7. Correlation matrix",
        title_x = 0.5,
        title_y = 0.95,
        title_font_size = 18,   
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgb(232,232,232)",
        width = 800,
        height = 400)

fig7.show()
